# PyBADS Example 3: Noisy objective function

In this example, we will show how to run PyBADS on a noisy target.

This notebook is Part 3 of a series of notebooks in which we present various example usages for BADS with the PyBADS package.

In [1]:
import numpy as np
from pybads.bads.bads import BADS

## 0. Noisy optimization

PyBADS is able to optimize also *noisy* objective functions. A noisy (or stochastic) objective function is an objective that will return different results if evaluated twice at the same point $\mathbf{x}$. Conversely, a non-noisy objective function is known as noiseless or deterministic. For example, noisy objectives are common in model fitting when the model is evaluated through simulation (e.g., via sampling aka Monte Carlo methods).

For a noisy objective, PyBADS attempts to minimize the *expected value* of $f(\mathbf{x})$,
$$
\mathbf{x}^\star = \arg\min_{\mathbf{x} \in \mathcal{X} \subseteq \mathbb{R}^D} \mathbb{E}\left[f(\mathbf{x})\right].
$$

## 1. Problem setup

For this example, we take as target a quadratic function and we add i.i.d. Gaussian noise to it (*noisy sphere*). In a real case, the noise would arise from some stochastic process in the calculation of the target.

We also set here several `options` for the optimization:

- We tell `bads` that the target is noisy by activating the `uncertainty_handling` option. This is not strictly needed, as `bads` can automatically detect if a target is noisy, but it is good practice to specify.
- We also limit the number of function evaluations with `max_fun_evals`, knowing that this is a simple example. Generally, `bads` will tend to run for longer on noisy problems to better explore the noisy landscape.
- Finally, we tell `bads` to re-evaluate the target at the returned solution with 100 samples via `noise_final_samples` (by default, `noise_final_samples = 10`, but since our function is inexpensive we can use more evaluations). These evaluations count towards the total budget of function evaluations `max_fun_evals`.

In [2]:
def noisy_sphere(x,sigma=1.0):
    """Simple quadratic function with added noise."""
    x_2d = np.atleast_2d(x)
    f = np.sum(x_2d**2, axis=1)
    noise = sigma*np.random.normal(size=x_2d.shape[0])
    return f + noise

x0 = np.array([[-3, -3]]);      # Starting point
lb = np.array([[-5, -5]])       # Lower bounds
ub = np.array([[5, 5]])         # Upper bounds
plb = np.array([[-2, -2]])      # Plausible lower bounds
pub = np.array([[2, 2]])        # Plausible upper bounds

options = {
    "uncertainty_handling": True,
    "max_fun_evals": 300,
    "noise_final_samples": 100
}

## 2. Run the optimization

We run `bads` with the user-defined `options`.

In [3]:
bads = BADS(noisy_sphere, x0, lb, ub, plb, pub, options=options)
optimize_result = bads.optimize()

Beginning optimization of a STOCHASTIC objective function

 Iteration f-count     E[f(x)]     SD[f(x)]     MeshScale     Method     Actions
     0         1      17.043956           nan      1.000000            
     0        33      -1.260743           nan      1.000000     Initial mesh       Initial points
     0        37      -1.260743      1.000000      0.500000     Refine grid       Train
     1        45      -1.260743      1.000000      0.250000     Refine grid       Train
     2        53      -0.252820      0.409687      0.125000     Refine grid       Train
     3        54      -0.011865      0.372887      0.125000     Successful search (('ES-ell', 1))       
     3        59      -0.059024      0.303255      0.125000     Successful search (('ES-ell', 1))       
     3        63      -0.097251      0.265894      0.125000     Incremental search (('ES-wcm', 1))       
     3        64      -0.099978      0.269720      0.125000     Incremental search (('ES-wcm', 1))       
    

## 3. Results and conclusions

First, note that in this case `optimize_result['fval']` is the *estimated* function value at `optimize_result['x']`, obtained by taking the mean of `options['noise_final_samples']` target evaluations (`noise_final_samples = 10` by default, but here we used 100). The uncertainty of the value of the function at the returned solution, `optimize_result['fsd']`, is the [standard error](https://en.wikipedia.org/wiki/Standard_error) of the mean.

If needed, the final samples used to estimate `fval` and `fsd` can be found in `optimize_result['yval_vec']`.

In [4]:
x_min = optimize_result['x']
fval = optimize_result['fval']
fsd = optimize_result['fsd']

print(f"BADS minimum at: x_min = {x_min.flatten()}, fval (estimated) = {fval:.4g} +/- {fsd:.2g}")
print(f"total f-count: {optimize_result['func_count']}, time: {round(optimize_result['total_time'], 2)} s")
print(f"final evaluations (shape): {optimize_result['yval_vec'].shape}")

BADS minimum at: x_min = [0.06634474 0.10446787], fval (estimated) = -0.1121 +/- 0.093
total f-count: 238, time: 3.69 s
final evaluations (shape): (100,)


We can also check the ground-truth value of the target function at the returned point once we remove the noise:

In [5]:
print(f"The true, noiseless value of f(x_min) is {noisy_sphere(x_min,sigma=0)[0]:.3g}.")

The true, noiseless value of f(x_min) is 0.0153.


Compare this to the true global minimum of the sphere function at $\textbf{x}^\star = [0,0]$, where $f^\star = 0$.

### Remarks
    
- While PyBADS can handle noisy targets, it cannot handle arbitrarily large noise.

- PyBADS will work best if the *standard deviation* of the objective function $\sigma$, when evaluated in the vicinity of the global solution, is small with respect to changes in the objective function itself (that is, there is a good signal-to-noise ratio). In many cases, $\sigma \approx 1$ or less should work (this is the default assumption). If you approximately know the magnitude of the noise in the vicinity of the solution, you can help BADS by specifying it in advance (set `options["noise_size"] = sigma_est`, where `sigma_est` is your estimate of the standard deviation).

- If the noise around the solution is too large, PyBADS will perform poorly. In that case, we recommend to increase the precision of your computation of the objective (e.g., by drawing more Monte Carlo samples) such that $\sigma \approx 1$ or even lower, as needed by your problem. Note that the noise farther away from the solution can be larger, and this is usually okay.

- In this example, we assumed the amount of noise in each target evaluation is unknown. If instead you *can* estimate the magnitude of the noise for each evaluation, see the next example.

- For more information on optimizing noisy objective functions, see the BADS wiki: https://github.com/acerbilab/bads/wiki#noisy-objective-function (this link points to the MATLAB wiki, but many of the questions and answers apply to PyBADS as well).


## Example 3: Full code

See [here](./src/pybads_example_3_noisy_objective.py) for a Python file with the code used in this example, with no extra fluff.